In [2]:
!pip install playsound==1.2.2



[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: C:\Users\hp\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [8]:
!pip install gtts


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: C:\Users\hp\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



     ---------------------------------------- 0.0/98.2 kB ? eta -:--:--
     ---------------------------------------- 98.2/98.2 kB 2.8 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.2.1
    Uninstalling click-8.2.1:
      Successfully uninstalled click-8.2.1


In [1]:
import cv2
import dlib
import time
import os
import numpy as np
import pandas as pd
from scipy.spatial import distance as dist
from imutils import face_utils
from gtts import gTTS   
import playsound
import random
import datetime


# 1. Setup Paths

os.chdir(r"C:\Users\hp\Desktop\Driver")  

PREDICTOR_PATH = "shape_predictor_68_face_landmarks.dat"
if not os.path.exists(PREDICTOR_PATH):
    raise FileNotFoundError(f" Missing {PREDICTOR_PATH}")


# 2. EAR & MAR Functions

def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    return (A + B) / (2.0 * C)

def mouth_aspect_ratio(mouth):
    A = dist.euclidean(mouth[2], mouth[10])  # 51, 59
    B = dist.euclidean(mouth[4], mouth[8])   # 53, 57
    C = dist.euclidean(mouth[0], mouth[6])   # 49, 55
    return (A + B) / (2.0 * C)


# 3. GenAI Voice Alert

def genai_alert(text="⚠️ Please wake up! You look drowsy."):
    filename = f"alert_{random.randint(0,9999)}.mp3"
    tts = gTTS(text=text, lang="en")
    tts.save(filename)
    playsound.playsound(filename)
    os.remove(filename)


# 4. Drowsiness Logger

log_data = []

def log_event(event_type, timestamp, ear_value=None, mar_value=None):
    log_data.append({
        "Time": timestamp,
        "Event": event_type,
        "EAR": ear_value,
        "MAR": mar_value
    })


# 5. Load Models

print("[INFO] Loading face detector and landmark predictor...")
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(PREDICTOR_PATH)

(lstart, lend) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rstart, rend) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
(mstart, mend) = face_utils.FACIAL_LANDMARKS_IDXS["mouth"]


# 6. Parameters

EAR_THRESHOLD = 0.25
EAR_CONSEC_FRAMES = 20  # drowsiness threshold
MAR_THRESHOLD = 0.6     # yawning threshold (tune this value)

COUNTER = 0
ALARM_ON = False


# 7. Start Video Capture

print("[INFO] Starting video stream...")
cap = cv2.VideoCapture(0)
time.sleep(1.0)

start_time = time.time()

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        rects = detector(gray, 0)

        for rect in rects:
            shape = predictor(gray, rect)
            shape = face_utils.shape_to_np(shape)

            leftEye = shape[lstart:lend]
            rightEye = shape[rstart:rend]
            mouth = shape[mstart:mend]

            leftEAR = eye_aspect_ratio(leftEye)
            rightEAR = eye_aspect_ratio(rightEye)
            ear = (leftEAR + rightEAR) / 2.0
            mar = mouth_aspect_ratio(mouth)

            # Draw contours
            leftHull = cv2.convexHull(leftEye)
            rightHull = cv2.convexHull(rightEye)
            mouthHull = cv2.convexHull(mouth)
            cv2.drawContours(frame, [leftHull], -1, (0, 255, 0), 1)
            cv2.drawContours(frame, [rightHull], -1, (0, 255, 0), 1)
            cv2.drawContours(frame, [mouthHull], -1, (0, 255, 0), 1)

            # EAR check (eyes closed → drowsiness)
            if ear < EAR_THRESHOLD:
                COUNTER += 1
                if COUNTER >= EAR_CONSEC_FRAMES and not ALARM_ON:
                    ALARM_ON = True
                    timestamp = round(time.time() - start_time, 2)
                    log_event("Drowsiness Alert", timestamp, ear, mar)
                    print("⚠️ ALERT: Drowsiness detected!")
                    genai_alert("⚠️ You seem very sleepy, please take a break!")

                    cv2.putText(frame, "DROWSINESS ALERT!", (10, 30),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            else:
                COUNTER = 0
                ALARM_ON = False

            # MAR check (yawning)
            if mar > MAR_THRESHOLD:
                timestamp = round(time.time() - start_time, 2)
                log_event("Yawn Alert", timestamp, ear, mar)
                print("😮 ALERT: Yawning detected!")
                genai_alert("😮 You are yawning. Please stay alert!")

                cv2.putText(frame, "YAWN ALERT!", (10, 60),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 165, 255), 2)

            # Display EAR and MAR
            cv2.putText(frame, f"EAR: {ear:.2f}", (300, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            cv2.putText(frame, f"MAR: {mar:.2f}", (300, 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

        cv2.imshow("Driver Drowsiness Detection", frame)

        key = cv2.waitKey(1) & 0xFF
        if key == ord("q"):
            break

finally:
    cap.release()
    cv2.destroyAllWindows()


# 8. Save Logs

df = pd.DataFrame(log_data)
log_file = "drowsiness_log.csv"
df.to_csv(log_file, index=False)
print(f"[INFO] Log saved → {log_file}")

# 9. GenAI Trip Summary

if not df.empty:
    num_drowsy = df[df["Event"] == "Drowsiness Alert"].shape[0]
    num_yawns = df[df["Event"] == "Yawn Alert"].shape[0]
    total_time = round(time.time() - start_time, 2)

    summary = (
        f"Driving session lasted {total_time} seconds. "
        f"Drowsiness detected {num_drowsy} times, "
        f"Yawning detected {num_yawns} times. "
    )
    if num_drowsy == 0 and num_yawns == 0:
        summary += "Excellent alertness throughout the trip."
    elif num_drowsy <= 2 and num_yawns <= 2:
        summary += "Mild drowsiness detected. Consider short breaks."
    else:
        summary += "High drowsiness detected. Strongly recommended to rest."

    print("\n===== 📋 Trip Summary =====")
    print(summary)
    print("===========================")

    # Speak the summary
    genai_alert(summary)
else:
    print("No drowsiness or yawning events recorded.")


[INFO] Loading face detector and landmark predictor...
[INFO] Starting video stream...
[INFO] Log saved → drowsiness_log.csv
No drowsiness or yawning events recorded.


In [14]:
# =========================================
# 🚗 Driver Drowsiness Detection + Fast GenAI + RAG
# =========================================

import cv2
import time
import os
import numpy as np
import pandas as pd
from scipy.spatial import distance as dist
from imutils import face_utils
from gtts import gTTS
import playsound
import random
import threading

# -------------------------
# 1️⃣ Setup Paths
# -------------------------
os.chdir(r"C:\Users\hp\Desktop\Driver")  # adjust path

PREDICTOR_PATH = "shape_predictor_68_face_landmarks.dat"
if not os.path.exists(PREDICTOR_PATH):
    raise FileNotFoundError(f"❌ Missing {PREDICTOR_PATH}")

# -------------------------
# 2️⃣ EAR & MAR Functions
# -------------------------
def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    return (A + B) / (2.0 * C)

def mouth_aspect_ratio(mouth):
    A = dist.euclidean(mouth[2], mouth[10])
    B = dist.euclidean(mouth[4], mouth[8])
    C = dist.euclidean(mouth[0], mouth[6])
    return (A + B) / (2.0 * C)

# -------------------------
# 3️⃣ GenAI TTS Alert
# -------------------------
def genai_alert(text="⚠️ Please wake up!"):
    filename = f"alert_{text[:10].replace(' ', '_')}.mp3"
    if not os.path.exists(filename):
        tts = gTTS(text=text, lang="en")
        tts.save(filename)
    threading.Thread(target=playsound.playsound, args=(filename,), daemon=True).start()

# -------------------------
# 4️⃣ Drowsiness Logger
# -------------------------
log_data = []

def log_event(event_type, timestamp, ear_value=None, mar_value=None):
    log_data.append({"Time": timestamp, "Event": event_type, "EAR": ear_value, "MAR": mar_value})

# -------------------------
# 5️⃣ Load Models
# -------------------------
import dlib
print("[INFO] Loading face detector and landmark predictor...")
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(PREDICTOR_PATH)
(lstart, lend) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rstart, rend) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
(mstart, mend) = face_utils.FACIAL_LANDMARKS_IDXS["mouth"]

# -------------------------
# 6️⃣ Parameters
# -------------------------
EAR_THRESHOLD = 0.25
EAR_CONSEC_FRAMES = 15
MAR_THRESHOLD = 0.6
COUNTER = 0
ALARM_ON = False
frame_count = 0

# -------------------------
# 7️⃣ Video Capture (Reduced Size)
# -------------------------
print("[INFO] Starting video stream...")
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 320)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 240)
time.sleep(1.0)
start_time = time.time()

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        rects = detector(gray, 0)

        # Process every 2nd frame for speed
        if frame_count % 2 == 0:
            for rect in rects:
                shape = predictor(gray, rect)
                shape = face_utils.shape_to_np(shape)
                leftEye = shape[lstart:lend]
                rightEye = shape[rstart:rend]
                mouth = shape[mstart:mend]

                leftEAR = eye_aspect_ratio(leftEye)
                rightEAR = eye_aspect_ratio(rightEye)
                ear = (leftEAR + rightEAR) / 2.0
                mar = mouth_aspect_ratio(mouth)

                leftHull = cv2.convexHull(leftEye)
                rightHull = cv2.convexHull(rightEye)
                mouthHull = cv2.convexHull(mouth)
                cv2.drawContours(frame, [leftHull], -1, (0, 255, 0), 1)
                cv2.drawContours(frame, [rightHull], -1, (0, 255, 0), 1)
                cv2.drawContours(frame, [mouthHull], -1, (0, 255, 0), 1)

                # Drowsiness check
                if ear < EAR_THRESHOLD:
                    COUNTER += 1
                    if COUNTER >= EAR_CONSEC_FRAMES and not ALARM_ON:
                        ALARM_ON = True
                        timestamp = round(time.time() - start_time, 2)
                        log_event("Drowsiness Alert", timestamp, ear, mar)
                        print("⚠️ ALERT: Drowsiness detected!")
                        genai_alert("⚠️ You seem sleepy! Take a break!")
                        cv2.putText(frame, "DROWSINESS ALERT!", (10, 30),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                else:
                    COUNTER = 0
                    ALARM_ON = False

                # Yawning check
                if mar > MAR_THRESHOLD:
                    timestamp = round(time.time() - start_time, 2)
                    log_event("Yawn Alert", timestamp, ear, mar)
                    print("😮 ALERT: Yawning detected!")
                    genai_alert("😮 You are yawning. Stay alert!")
                    cv2.putText(frame, "YAWN ALERT!", (10, 60),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 165, 255), 2)

                # Display EAR/MAR
                cv2.putText(frame, f"EAR: {ear:.2f}", (200, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
                cv2.putText(frame, f"MAR: {mar:.2f}", (200, 60),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

        cv2.imshow("Driver Drowsiness Detection", frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

finally:
    cap.release()
    cv2.destroyAllWindows()

# -------------------------
# 8️⃣ Save Logs
# -------------------------
df = pd.DataFrame(log_data)
df.to_csv("drowsiness_log_fast.csv", index=False)
print(f"[INFO] Log saved → drowsiness_log_fast.csv")

# -------------------------
# 9️⃣ GenAI Trip Summary
# -------------------------
if not df.empty:
    num_drowsy = df[df["Event"] == "Drowsiness Alert"].shape[0]
    num_yawns = df[df["Event"] == "Yawn Alert"].shape[0]
    total_time = round(time.time() - start_time, 2)
    summary = (f"Driving session lasted {total_time} sec. "
               f"Drowsiness detected {num_drowsy} times, "
               f"Yawning detected {num_yawns} times.")
    print("\n===== 📋 Trip Summary =====")
    print(summary)
    genai_alert(summary)
else:
    print("No events recorded.")

# -------------------------
# 🔹 Fast Offline RAG Setup
# -------------------------
try:
    from langchain.chains import RetrievalQA
    from langchain.vectorstores import Chroma
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    from langchain.document_loaders import DirectoryLoader
    from langchain.llms import HuggingFacePipeline
    from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
    import torch

    # Load precomputed vectorstore (saved earlier)
    vectorstore = Chroma(persist_directory="vectorstore")  # folder must exist
    retriever = vectorstore.as_retriever()

    # Smaller local model for fast inference
    model_name = r"C:\Users\hp\Desktop\Driver\distilgpt2"  # example path
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)
    pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_length=128)
    local_llm = HuggingFacePipeline(pipeline=pipe)
    offline_qa_chain = RetrievalQA.from_chain_type(llm=local_llm, retriever=retriever, return_source_documents=True)

    def rag_advice_offline(query):
        try:
            return offline_qa_chain.run(query)
        except:
            return "Unable to fetch advice right now."

    print("[INFO] Fast Offline RAG ready!")
except Exception as e:
    print(f"[WARNING] Fast RAG setup failed: {e}")


[INFO] Loading face detector and landmark predictor...
[INFO] Starting video stream...


KeyboardInterrupt: 

In [15]:
# =========================================
# 🚗 Driver Drowsiness Detection + Fast GenAI + Threaded RAG
# =========================================

import cv2
import time
import os
import numpy as np
import pandas as pd
from scipy.spatial import distance as dist
from imutils import face_utils
from gtts import gTTS
import playsound
import random
import threading

# -------------------------
# 1️⃣ Setup Paths
# -------------------------
os.chdir(r"C:\Users\hp\Desktop\Driver")  # adjust path
PREDICTOR_PATH = "shape_predictor_68_face_landmarks.dat"
if not os.path.exists(PREDICTOR_PATH):
    raise FileNotFoundError(f"❌ Missing {PREDICTOR_PATH}")

# -------------------------
# 2️⃣ EAR & MAR Functions
# -------------------------
def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    return (A + B) / (2.0 * C)

def mouth_aspect_ratio(mouth):
    A = dist.euclidean(mouth[2], mouth[10])
    B = dist.euclidean(mouth[4], mouth[8])
    C = dist.euclidean(mouth[0], mouth[6])
    return (A + B) / (2.0 * C)

# -------------------------
# 3️⃣ GenAI TTS Alert
# -------------------------
def genai_alert(text="⚠️ Please wake up!"):
    filename = f"alert_{text[:10].replace(' ', '_')}.mp3"
    if not os.path.exists(filename):
        tts = gTTS(text=text, lang="en")
        tts.save(filename)
    threading.Thread(target=playsound.playsound, args=(filename,), daemon=True).start()

# -------------------------
# 4️⃣ Drowsiness Logger
# -------------------------
log_data = []

def log_event(event_type, timestamp, ear_value=None, mar_value=None):
    log_data.append({"Time": timestamp, "Event": event_type, "EAR": ear_value, "MAR": mar_value})

# -------------------------
# 5️⃣ Load Models
# -------------------------
import dlib
print("[INFO] Loading face detector and landmark predictor...")
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(PREDICTOR_PATH)
(lstart, lend) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rstart, rend) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
(mstart, mend) = face_utils.FACIAL_LANDMARKS_IDXS["mouth"]

# -------------------------
# 6️⃣ Parameters
# -------------------------
EAR_THRESHOLD = 0.25
EAR_CONSEC_FRAMES = 15
MAR_THRESHOLD = 0.6
COUNTER = 0
ALARM_ON = False
frame_count = 0

# -------------------------
# 7️⃣ Video Capture (Reduced Size)
# -------------------------
print("[INFO] Starting video stream...")
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 320)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 240)
time.sleep(1.0)
start_time = time.time()

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        rects = detector(gray, 0)

        # Process every 2nd frame for speed
        if frame_count % 2 == 0:
            for rect in rects:
                shape = predictor(gray, rect)
                shape = face_utils.shape_to_np(shape)
                leftEye = shape[lstart:lend]
                rightEye = shape[rstart:rend]
                mouth = shape[mstart:mend]

                leftEAR = eye_aspect_ratio(leftEye)
                rightEAR = eye_aspect_ratio(rightEye)
                ear = (leftEAR + rightEAR) / 2.0
                mar = mouth_aspect_ratio(mouth)

                leftHull = cv2.convexHull(leftEye)
                rightHull = cv2.convexHull(rightEye)
                mouthHull = cv2.convexHull(mouth)
                cv2.drawContours(frame, [leftHull], -1, (0, 255, 0), 1)
                cv2.drawContours(frame, [rightHull], -1, (0, 255, 0), 1)
                cv2.drawContours(frame, [mouthHull], -1, (0, 255, 0), 1)

                # Drowsiness check
                if ear < EAR_THRESHOLD:
                    COUNTER += 1
                    if COUNTER >= EAR_CONSEC_FRAMES and not ALARM_ON:
                        ALARM_ON = True
                        timestamp = round(time.time() - start_time, 2)
                        log_event("Drowsiness Alert", timestamp, ear, mar)
                        print("⚠️ ALERT: Drowsiness detected!")
                        genai_alert("⚠️ You seem sleepy! Take a break!")
                        cv2.putText(frame, "DROWSINESS ALERT!", (10, 30),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                else:
                    COUNTER = 0
                    ALARM_ON = False

                # Yawning check
                if mar > MAR_THRESHOLD:
                    timestamp = round(time.time() - start_time, 2)
                    log_event("Yawn Alert", timestamp, ear, mar)
                    print("😮 ALERT: Yawning detected!")
                    genai_alert("😮 You are yawning. Stay alert!")
                    cv2.putText(frame, "YAWN ALERT!", (10, 60),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 165, 255), 2)

                # Display EAR/MAR
                cv2.putText(frame, f"EAR: {ear:.2f}", (200, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
                cv2.putText(frame, f"MAR: {mar:.2f}", (200, 60),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

        cv2.imshow("Driver Drowsiness Detection", frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

finally:
    cap.release()
    cv2.destroyAllWindows()

# -------------------------
# 8️⃣ Save Logs
# -------------------------
df = pd.DataFrame(log_data)
df.to_csv("drowsiness_log_fast.csv", index=False)
print(f"[INFO] Log saved → drowsiness_log_fast.csv")

# -------------------------
# 9️⃣ GenAI Trip Summary
# -------------------------
if not df.empty:
    num_drowsy = df[df["Event"] == "Drowsiness Alert"].shape[0]
    num_yawns = df[df["Event"] == "Yawn Alert"].shape[0]
    total_time = round(time.time() - start_time, 2)
    summary = (f"Driving session lasted {total_time} sec. "
               f"Drowsiness detected {num_drowsy} times, "
               f"Yawning detected {num_yawns} times.")
    print("\n===== 📋 Trip Summary =====")
    print(summary)
    genai_alert(summary)
else:
    print("No events recorded.")

# -------------------------
# 🔹 Threaded Fast Offline RAG Setup
# -------------------------
def init_rag():
    global offline_qa_chain
    try:
        from langchain.chains import RetrievalQA
        from langchain.vectorstores import Chroma
        from langchain.llms import HuggingFacePipeline
        from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

        vectorstore = Chroma(persist_directory="vectorstore")  # must exist
        retriever = vectorstore.as_retriever()

        model_name = r"C:\Users\hp\Desktop\Driver\distilgpt2"  # example path
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForCausalLM.from_pretrained(model_name)
        pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_length=128)
        local_llm = HuggingFacePipeline(pipeline=pipe)

        offline_qa_chain = RetrievalQA.from_chain_type(
            llm=local_llm,
            retriever=retriever,
            return_source_documents=True
        )
        print("[INFO] Fast Offline RAG ready!")
    except Exception as e:
        print(f"[WARNING] Fast RAG setup failed: {e}")

# Launch RAG in background
threading.Thread(target=init_rag, daemon=True).start()


[INFO] Loading face detector and landmark predictor...
[INFO] Starting video stream...
😮 ALERT: Yawning detected!


Exception in thread Thread-7 (_playsoundWin):
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "C:\Users\hp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\ipykernel\ipkernel.py", line 772, in run_closure
    _threading_Thread_run(self)
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\hp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\playsound.py", line 35, in _playsoundWin
Exception in thread Thread-8 (_playsoundWin):
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.

😮 ALERT: Yawning detected!
😮 ALERT: Yawning detected!
😮 ALERT: Yawning detected!
😮 ALERT: Yawning detected!


Thread-11 (_playsoundWin):
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "C:\Users\hp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\ipykernel\ipkernel.py", line 772, in run_closure
    _threading_Thread_run(self)
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\hp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\playsound.py", line 35, in _playsoundWin
    winCommand('open "' + sound + '" alias', alias)
  File "C:\Users\hp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages

😮 ALERT: Yawning detected!
😮 ALERT: Yawning detected!
😮 ALERT: Yawning detected!
😮 ALERT: Yawning detected!


...

😮 ALERT: Yawning detected!


Exception in thread Thread-65 (_playsoundWin):
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "C:\Users\hp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\ipykernel\ipkernel.py", line 772, in run_closure
    _threading_Thread_run(self)
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\hp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\playsound.py", line 35, in _playsoundWin
    winCommand('open "' + sound + '" alias', alias)
  File "C:\Users\hp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\Local

😮 ALERT: Yawning detected!
[INFO] Log saved → drowsiness_log_fast.csv

===== 📋 Trip Summary =====
Driving session lasted 36.7 sec. Drowsiness detected 1 times, Yawning detected 58 times.


C:\Users\hp\AppData\Local\Temp\ipykernel_24532\1271510872.py:192: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(persist_directory="vectorstore")  # must exist


[WARNING] Fast RAG setup failed: Repo id must use alphanumeric chars or '-', '_', '.', '--' and '..' are forbidden, '-' and '.' cannot start or end the name, max length is 96: 'C:\Users\hp\Desktop\Driver\distilgpt2'.


In [16]:
# ================================
# 🚗 Driver Drowsiness Detection + GPU GenAI + Offline RAG + Voice Query
# ================================

import cv2
import dlib
import time
import os
import numpy as np
import pandas as pd
from scipy.spatial import distance as dist
from imutils import face_utils
from gtts import gTTS
import playsound
import threading
import random
import speech_recognition as sr

# ================================
# 1️⃣ Setup Paths
# ================================
os.chdir(r"C:\Users\hp\Desktop\Driver")
PREDICTOR_PATH = "shape_predictor_68_face_landmarks.dat"
if not os.path.exists(PREDICTOR_PATH):
    raise FileNotFoundError(f"❌ Missing {PREDICTOR_PATH}")

# ================================
# 2️⃣ EAR & MAR Functions
# ================================
def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    return (A + B) / (2.0 * C)

def mouth_aspect_ratio(mouth):
    A = dist.euclidean(mouth[2], mouth[10])
    B = dist.euclidean(mouth[4], mouth[8])
    C = dist.euclidean(mouth[0], mouth[6])
    return (A + B) / (2.0 * C)

# ================================
# 3️⃣ Threaded GenAI Voice Alert
# ================================
def genai_alert(text):
    def _speak():
        filename = f"alert_{random.randint(0,9999)}.mp3"
        tts = gTTS(text=text, lang="en")
        tts.save(filename)
        playsound.playsound(filename)
        os.remove(filename)
    threading.Thread(target=_speak, daemon=True).start()

# ================================
# 4️⃣ Drowsiness Logger
# ================================
log_data = []
def log_event(event_type, timestamp, ear=None, mar=None):
    log_data.append({
        "Time": timestamp,
        "Event": event_type,
        "EAR": ear,
        "MAR": mar
    })

# ================================
# 5️⃣ Load Models
# ================================
print("[INFO] Loading face detector and landmark predictor...")
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(PREDICTOR_PATH)
(lstart, lend) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rstart, rend) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
(mstart, mend) = face_utils.FACIAL_LANDMARKS_IDXS["mouth"]

# ================================
# 6️⃣ Parameters
# ================================
EAR_THRESHOLD = 0.25
EAR_CONSEC_FRAMES = 20
MAR_THRESHOLD = 0.6
COUNTER = 0
ALARM_ON = False
drowsy_count = 0
yawn_count = 0

# ================================
# 7️⃣ Offline RAG Setup (GPU)
# ================================
def init_rag():
    global offline_qa_chain
    try:
        from langchain.chains import RetrievalQA
        from langchain.vectorstores import Chroma
        from langchain.text_splitter import RecursiveCharacterTextSplitter
        from langchain.document_loaders import DirectoryLoader
        from langchain.llms import HuggingFacePipeline
        from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
        import torch

        # Load documents
        loader = DirectoryLoader("knowledge_base", glob="**/*.txt")
        docs = loader.load()
        splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        chunks = splitter.split_documents(docs)

        # Embeddings + vectorstore
        from langchain.embeddings import HuggingFaceEmbeddings
        embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
        vectorstore = Chroma.from_documents(chunks, embeddings)
        retriever = vectorstore.as_retriever()

        # Load LLaMA2 on GPU
        model_name = r"C:\Users\hp\Desktop\Driver\llama2_7b"
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForCausalLM.from_pretrained(
            model_name, torch_dtype=torch.float16, device_map="auto"
        )
        pipe = pipeline(
            "text-generation",
            model=model,
            tokenizer=tokenizer,
            device=0,           # GPU
            max_length=256,
            temperature=0.5
        )
        local_llm = HuggingFacePipeline(pipeline=pipe)

        offline_qa_chain = RetrievalQA.from_chain_type(
            llm=local_llm,
            retriever=retriever,
            return_source_documents=True
        )
        print("[INFO] Offline RAG ready on GPU!")
    except Exception as e:
        print(f"[WARNING] Offline RAG setup failed: {e}")
        offline_qa_chain = None

# Start RAG in background
threading.Thread(target=init_rag, daemon=True).start()

def rag_advice_offline(query):
    if offline_qa_chain:
        return offline_qa_chain.run(query)
    return "Offline RAG not available."

# ================================
# 8️⃣ Voice Query Thread
# ================================
def listen_for_query():
    r = sr.Recognizer()
    mic = sr.Microphone()
    while True:
        try:
            with mic as source:
                r.adjust_for_ambient_noise(source)
                print("[INFO] Say 'hey system' to ask a question...")
                audio = r.listen(source, timeout=5)
            text = r.recognize_google(audio).lower()
            if "hey system" in text:
                print("[INFO] Listening for your query...")
                with mic as source:
                    r.adjust_for_ambient_noise(source)
                    audio = r.listen(source, timeout=10)
                query = r.recognize_google(audio)
                print(f"[QUERY] {query}")
                answer = rag_advice_offline(query)
                print(f"[ANSWER] {answer}")
                genai_alert(answer)
        except Exception:
            continue

threading.Thread(target=listen_for_query, daemon=True).start()

# ================================
# 9️⃣ Video Capture
# ================================
print("[INFO] Starting video stream...")
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 320)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 240)
time.sleep(1.0)
start_time = time.time()

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        rects = detector(gray, 0)

        for rect in rects:
            shape = predictor(gray, rect)
            shape = face_utils.shape_to_np(shape)

            leftEye = shape[lstart:lend]
            rightEye = shape[rstart:rend]
            mouth = shape[mstart:mend]

            leftEAR = eye_aspect_ratio(leftEye)
            rightEAR = eye_aspect_ratio(rightEye)
            ear = (leftEAR + rightEAR) / 2.0
            mar = mouth_aspect_ratio(mouth)

            # Draw contours
            cv2.drawContours(frame, [cv2.convexHull(leftEye),
                                     cv2.convexHull(rightEye),
                                     cv2.convexHull(mouth)], -1, (0, 255, 0), 1)

            # Drowsiness detection
            if ear < EAR_THRESHOLD:
                COUNTER += 1
                if COUNTER >= EAR_CONSEC_FRAMES and not ALARM_ON:
                    ALARM_ON = True
                    timestamp = round(time.time() - start_time, 2)
                    drowsy_count += 1
                    log_event("Drowsiness Alert", timestamp, ear, mar)
                    genai_alert(f"⚠️ You seem very sleepy! Alert count: {drowsy_count}")
                    cv2.putText(frame, f"DROWSINESS ALERT! Count: {drowsy_count}", (10, 30),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
            else:
                COUNTER = 0
                ALARM_ON = False

            # Yawning detection
            if mar > MAR_THRESHOLD:
                timestamp = round(time.time() - start_time, 2)
                yawn_count += 1
                log_event("Yawn Alert", timestamp, ear, mar)
                genai_alert(f"😮 Yawning detected! Count: {yawn_count}")
                cv2.putText(frame, f"YAWN ALERT! Count: {yawn_count}", (10, 60),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 165, 255), 2)

            # Display EAR/MAR
            cv2.putText(frame, f"EAR: {ear:.2f}", (250, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            cv2.putText(frame, f"MAR: {mar:.2f}", (250, 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

        cv2.imshow("Driver Drowsiness Detection", frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

finally:
    cap.release()
    cv2.destroyAllWindows()
    df = pd.DataFrame(log_data)
    df.to_csv("drowsiness_log.csv", index=False)
    print("[INFO] Log saved → drowsiness_log.csv")


[INFO] Loading face detector and landmark predictor...


Error loading file knowledge_base\safety_tips.txt.txt


[INFO] Starting video stream...
[WARNING] Offline RAG setup failed: unstructured package not found, please install it with `pip install unstructured`
[INFO] Say 'hey system' to ask a question...
[INFO] Say 'hey system' to ask a question...
[INFO] Say 'hey system' to ask a question...
[INFO] Say 'hey system' to ask a question...
[INFO] Listening for your query...
[QUERY] morning
[ANSWER] Offline RAG not available.
[INFO] Say 'hey system' to ask a question...
[INFO] Say 'hey system' to ask a question...
[INFO] Say 'hey system' to ask a question...
[INFO] Say 'hey system' to ask a question...
[INFO] Say 'hey system' to ask a question...
[INFO] Say 'hey system' to ask a question...
[INFO] Say 'hey system' to ask a question...
[INFO] Say 'hey system' to ask a question...
[INFO] Say 'hey system' to ask a question...
[INFO] Log saved → drowsiness_log.csv
[INFO] Say 'hey system' to ask a question...
[INFO] Say 'hey system' to ask a question...
[INFO] Say 'hey system' to ask a question...
[INFO

In [ ]:
# =========================================
# 🚗 Driver Drowsiness Detection + Yawning + TTS (pyttsx3) + Offline RAG + GPU + Voice Query
# =========================================

import cv2
import dlib
import time
import os
import numpy as np
import pandas as pd
from scipy.spatial import distance as dist
from imutils import face_utils
import threading
import torch

# --- Voice recognition ---
import speech_recognition as sr

# --- Text-to-Speech (offline, no file permission issues) ---
import pyttsx3
engine = pyttsx3.init()
engine.setProperty('rate', 150)

def genai_alert(text):
    """Speak alert text in a separate thread (non-blocking)"""
    def speak():
        engine.say(text)
        engine.runAndWait()
    threading.Thread(target=speak, daemon=True).start()

# ================================
# 1️⃣ Setup Paths
# ================================
os.chdir(r"C:\Users\hp\Desktop\Driver")
PREDICTOR_PATH = "shape_predictor_68_face_landmarks.dat"
if not os.path.exists(PREDICTOR_PATH):
    raise FileNotFoundError(f"❌ Missing {PREDICTOR_PATH}")

# ================================
# 2️⃣ EAR & MAR Functions
# ================================
def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    return (A + B) / (2.0 * C)

def mouth_aspect_ratio(mouth):
    A = dist.euclidean(mouth[2], mouth[10])
    B = dist.euclidean(mouth[4], mouth[8])
    C = dist.euclidean(mouth[0], mouth[6])
    return (A + B) / (2.0 * C)

# ================================
# 3️⃣ Drowsiness Logger
# ================================
log_data = []
def log_event(event_type, timestamp, ear_value=None, mar_value=None):
    log_data.append({
        "Time": timestamp,
        "Event": event_type,
        "EAR": ear_value,
        "MAR": mar_value
    })

# ================================
# 4️⃣ Load Models
# ================================
print("[INFO] Loading face detector and landmark predictor...")
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(PREDICTOR_PATH)
(lstart, lend) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rstart, rend) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
(mstart, mend) = face_utils.FACIAL_LANDMARKS_IDXS["mouth"]

# ================================
# 5️⃣ Parameters
# ================================
EAR_THRESHOLD = 0.25
EAR_CONSEC_FRAMES = 20
MAR_THRESHOLD = 0.6

COUNTER = 0
ALARM_ON = False
YAWN_COUNT = 0
DROWSY_COUNT = 0

# ================================
# 6️⃣ Offline RAG Setup (GPU)
# ================================
try:
    from langchain.chains import RetrievalQA
    from langchain.vectorstores import Chroma
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    from langchain.document_loaders import DirectoryLoader
    from langchain.llms import HuggingFacePipeline
    from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

    loader = DirectoryLoader("knowledge_base", glob="**/*.txt")
    docs = loader.load()
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = splitter.split_documents(docs)

    from langchain.embeddings import HuggingFaceEmbeddings
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vectorstore = Chroma.from_documents(chunks, embeddings)
    retriever = vectorstore.as_retriever()

    # GPU-enabled LLM
    model_name = r"C:\Users\hp\Desktop\Driver\llama2_7b"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16,
        device_map="auto"
    )
    pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_length=512, temperature=0.5, device=0)
    local_llm = HuggingFacePipeline(pipeline=pipe)
    offline_qa_chain = RetrievalQA.from_chain_type(
        llm=local_llm,
        retriever=retriever,
        return_source_documents=True
    )

    def rag_advice_offline(query):
        try:
            return offline_qa_chain.run(query)
        except:
            return "Unable to fetch offline advice."

    print("[INFO] Offline RAG ready!")

except Exception as e:
    print(f"[WARNING] Offline RAG setup failed: {e}")
    rag_advice_offline = lambda q: "Offline RAG not available."

# ================================
# 7️⃣ Voice Query Thread
# ================================
def listen_for_query():
    r = sr.Recognizer()
    mic = sr.Microphone()
    while True:
        try:
            print("[INFO] Listening for questions... Say 'hey system' to activate.")
            with mic as source:
                r.adjust_for_ambient_noise(source)
                audio = r.listen(source, timeout=5)
            text = r.recognize_google(audio)
            if "hey system" in text.lower():
                print("[INFO] Activated. Please ask your question...")
                with mic as source:
                    r.adjust_for_ambient_noise(source)
                    audio = r.listen(source, timeout=10)
                query = r.recognize_google(audio)
                print(f"[QUERY] {query}")
                answer = rag_advice_offline(query)
                print(f"[ANSWER] {answer}")
                genai_alert(answer)
        except Exception:
            pass

voice_thread = threading.Thread(target=listen_for_query, daemon=True)
voice_thread.start()

# ================================
# 8️⃣ Start Video Capture
# ================================
print("[INFO] Starting video stream...")
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 320)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 240)
time.sleep(1.0)
start_time = time.time()

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        rects = detector(gray, 0)

        for rect in rects:
            shape = predictor(gray, rect)
            shape = face_utils.shape_to_np(shape)

            leftEye = shape[lstart:lend]
            rightEye = shape[rstart:rend]
            mouth = shape[mstart:mend]

            leftEAR = eye_aspect_ratio(leftEye)
            rightEAR = eye_aspect_ratio(rightEye)
            ear = (leftEAR + rightEAR) / 2.0
            mar = mouth_aspect_ratio(mouth)

            leftHull = cv2.convexHull(leftEye)
            rightHull = cv2.convexHull(rightEye)
            mouthHull = cv2.convexHull(mouth)
            cv2.drawContours(frame, [leftHull, rightHull, mouthHull], -1, (0, 255, 0), 1)

            # Drowsiness detection
            if ear < EAR_THRESHOLD:
                COUNTER += 1
                if COUNTER >= EAR_CONSEC_FRAMES and not ALARM_ON:
                    ALARM_ON = True
                    DROWSY_COUNT += 1
                    timestamp = round(time.time() - start_time, 2)
                    log_event("Drowsiness Alert", timestamp, ear, mar)
                    print(f"⚠️ ALERT: Drowsiness detected! Count: {DROWSY_COUNT}")
                    genai_alert(f"⚠️ You seem very sleepy, take a break! This is alert number {DROWSY_COUNT}.")
                    cv2.putText(frame, f"DROWSINESS ALERT! Count: {DROWSY_COUNT}", (10, 30),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            else:
                COUNTER = 0
                ALARM_ON = False

            # Yawning detection
            if mar > MAR_THRESHOLD:
                YAWN_COUNT += 1
                timestamp = round(time.time() - start_time, 2)
                log_event("Yawn Alert", timestamp, ear, mar)
                print(f"😮 ALERT: Yawning detected! Count: {YAWN_COUNT}")
                genai_alert(f"😮 You are yawning. Stay alert! Yawn count: {YAWN_COUNT}")
                cv2.putText(frame, f"YAWN ALERT! Count: {YAWN_COUNT}", (10, 60),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 165, 255), 2)

            # Display EAR/MAR
            cv2.putText(frame, f"EAR: {ear:.2f}", (300, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            cv2.putText(frame, f"MAR: {mar:.2f}", (300, 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

        cv2.imshow("Driver Drowsiness Detection", frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

finally:
    cap.release()
    cv2.destroyAllWindows()

    df = pd.DataFrame(log_data)
    df.to_csv("drowsiness_log.csv", index=False)
    print("[INFO] Log saved → drowsiness_log.csv")

    total_time = round(time.time() - start_time, 2)
    print(f"\n===== 📋 Trip Summary =====")
    print(f"Duration: {total_time} sec | Drowsiness Alerts: {DROWSY_COUNT} | Yawn Alerts: {YAWN_COUNT}")
    genai_alert(f"Trip Summary: Duration {total_time} seconds. Drowsiness alerts {DROWSY_COUNT}. Yawning alerts {YAWN_COUNT}.")


[INFO] Loading face detector and landmark predictor...


libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.


[WARNING] Offline RAG setup failed: Repo id must use alphanumeric chars or '-', '_', '.', '--' and '..' are forbidden, '-' and '.' cannot start or end the name, max length is 96: 'C:\Users\hp\Desktop\Driver\llama2_7b'.
[INFO] Listening for questions... Say 'hey system' to activate.
[INFO] Starting video stream...
[INFO] Listening for questions... Say 'hey system' to activate.
[INFO] Listening for questions... Say 'hey system' to activate.
[INFO] Listening for questions... Say 'hey system' to activate.


In [ ]:
import cv2
import dlib
import pyttsx3
import threading
import time
from scipy.spatial import distance as dist

# ------------------ Thread-safe TTS ------------------
tts_engine = pyttsx3.init()
tts_engine.setProperty('rate', 160)
tts_lock = threading.Lock()

def genai_alert(text):
    def speak():
        with tts_lock:
            tts_engine.say(text)
            tts_engine.runAndWait()
    threading.Thread(target=speak, daemon=True).start()

# ------------------ Drowsiness / Yawn Detection ------------------
EYE_AR_THRESH = 0.25   # Eye aspect ratio threshold
EYE_AR_CONSEC_FRAMES = 20
MOUTH_AR_THRESH = 0.7  # Mouth aspect ratio threshold

def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

def mouth_aspect_ratio(mouth):
    A = dist.euclidean(mouth[13], mouth[19])
    B = dist.euclidean(mouth[14], mouth[18])
    C = dist.euclidean(mouth[12], mouth[16])
    mar = (A + B) / (2.0 * C)
    return mar

# ------------------ Load face detector and predictor ------------------
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

(lStart, lEnd) = (42, 48)  # Right eye
(rStart, rEnd) = (36, 42)  # Left eye
(mStart, mEnd) = (60, 68)  # Mouth

# ------------------ Video Capture ------------------
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

drowsy_count = 0
yawn_count = 0
frame_counter = 0

print("[INFO] Starting video stream...")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    rects = detector(gray, 0)

    for rect in rects:
        shape = predictor(gray, rect)
        shape = [(shape.part(i).x, shape.part(i).y) for i in range(68)]

        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]
        mouth = shape[mStart:mEnd]

        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)
        ear = (leftEAR + rightEAR) / 2.0

        mar = mouth_aspect_ratio(mouth)

        # Drowsiness detection
        if ear < EYE_AR_THRESH:
            frame_counter += 1
            if frame_counter >= EYE_AR_CONSEC_FRAMES:
                drowsy_count += 1
                genai_alert(f"⚠️ ALERT: Drowsiness detected! Count: {drowsy_count}")
                frame_counter = 0
        else:
            frame_counter = 0

        # Yawn detection
        if mar > MOUTH_AR_THRESH:
            yawn_count += 1
            genai_alert(f"😮 ALERT: Yawning detected! Count: {yawn_count}")

    cv2.imshow("Driver Monitoring", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [1]:
# =========================================
# 🚗 Driver Drowsiness Detection + Yawning + TTS (pyttsx3) + Voice Query
# =========================================

import cv2
import dlib
import time
import os
import numpy as np
import pandas as pd
from scipy.spatial import distance as dist
from imutils import face_utils
import threading

# --- Voice recognition ---
import speech_recognition as sr

# --- Text-to-Speech (offline) ---
import pyttsx3
engine = pyttsx3.init()
engine.setProperty('rate', 150)

def genai_alert(text):
    """Speak alert text in a separate thread (non-blocking)"""
    def speak():
        engine.say(text)
        engine.runAndWait()
    threading.Thread(target=speak, daemon=True).start()

# ================================
# 1️⃣ Setup Paths
# ================================
os.chdir(r"C:\Users\hp\Desktop\Driver")  # Change to your folder
PREDICTOR_PATH = "shape_predictor_68_face_landmarks.dat"
if not os.path.exists(PREDICTOR_PATH):
    raise FileNotFoundError(f"❌ Missing {PREDICTOR_PATH}")

# ================================
# 2️⃣ EAR & MAR Functions
# ================================
def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    return (A + B) / (2.0 * C)

def mouth_aspect_ratio(mouth):
    A = dist.euclidean(mouth[2], mouth[10])
    B = dist.euclidean(mouth[4], mouth[8])
    C = dist.euclidean(mouth[0], mouth[6])
    return (A + B) / (2.0 * C)

# ================================
# 3️⃣ Drowsiness Logger
# ================================
log_data = []
def log_event(event_type, timestamp, ear_value=None, mar_value=None):
    log_data.append({
        "Time": timestamp,
        "Event": event_type,
        "EAR": ear_value,
        "MAR": mar_value
    })

# ================================
# 4️⃣ Load Models
# ================================
print("[INFO] Loading face detector and landmark predictor...")
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(PREDICTOR_PATH)
(lstart, lend) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rstart, rend) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
(mstart, mend) = face_utils.FACIAL_LANDMARKS_IDXS["mouth"]

# ================================
# 5️⃣ Parameters
# ================================
EAR_THRESHOLD = 0.25
EAR_CONSEC_FRAMES = 20
MAR_THRESHOLD = 0.6

COUNTER = 0
ALARM_ON = False
YAWN_COUNT = 0
DROWSY_COUNT = 0

# ================================
# 6️⃣ Voice Query Thread (placeholder)
# ================================
def listen_for_query():
    r = sr.Recognizer()
    mic = sr.Microphone()
    while True:
        try:
            print("[INFO] Listening for questions... Say 'hey system' to activate.")
            with mic as source:
                r.adjust_for_ambient_noise(source)
                audio = r.listen(source, timeout=5)
            text = r.recognize_google(audio)
            if "hey system" in text.lower():
                print("[INFO] Activated. Please ask your question...")
                with mic as source:
                    r.adjust_for_ambient_noise(source)
                    audio = r.listen(source, timeout=10)
                query = r.recognize_google(audio)
                print(f"[QUERY] {query}")
                answer = "This is a placeholder response."  # Replace with any offline logic if needed
                print(f"[ANSWER] {answer}")
                genai_alert(answer)
        except Exception:
            pass

voice_thread = threading.Thread(target=listen_for_query, daemon=True)
voice_thread.start()

# ================================
# 7️⃣ Start Video Capture
# ================================
print("[INFO] Starting video stream...")
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 320)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 240)
time.sleep(1.0)
start_time = time.time()

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        rects = detector(gray, 0)

        for rect in rects:
            shape = predictor(gray, rect)
            shape = face_utils.shape_to_np(shape)

            leftEye = shape[lstart:lend]
            rightEye = shape[rstart:rend]
            mouth = shape[mstart:mend]

            leftEAR = eye_aspect_ratio(leftEye)
            rightEAR = eye_aspect_ratio(rightEye)
            ear = (leftEAR + rightEAR) / 2.0
            mar = mouth_aspect_ratio(mouth)

            leftHull = cv2.convexHull(leftEye)
            rightHull = cv2.convexHull(rightEye)
            mouthHull = cv2.convexHull(mouth)
            cv2.drawContours(frame, [leftHull, rightHull, mouthHull], -1, (0, 255, 0), 1)

            # Drowsiness detection
            if ear < EAR_THRESHOLD:
                COUNTER += 1
                if COUNTER >= EAR_CONSEC_FRAMES and not ALARM_ON:
                    ALARM_ON = True
                    DROWSY_COUNT += 1
                    timestamp = round(time.time() - start_time, 2)
                    log_event("Drowsiness Alert", timestamp, ear, mar)
                    print(f"⚠️ ALERT: Drowsiness detected! Count: {DROWSY_COUNT}")
                    genai_alert(f"⚠️ You seem very sleepy, take a break! Alert number {DROWSY_COUNT}")
                    cv2.putText(frame, f"DROWSINESS ALERT! Count: {DROWSY_COUNT}", (10, 30),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            else:
                COUNTER = 0
                ALARM_ON = False

            # Yawning detection
            if mar > MAR_THRESHOLD:
                YAWN_COUNT += 1
                timestamp = round(time.time() - start_time, 2)
                log_event("Yawn Alert", timestamp, ear, mar)
                print(f"😮 ALERT: Yawning detected! Count: {YAWN_COUNT}")
                genai_alert(f"😮 You are yawning. Stay alert! Yawn count: {YAWN_COUNT}")
                cv2.putText(frame, f"YAWN ALERT! Count: {YAWN_COUNT}", (10, 60),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 165, 255), 2)

            # Display EAR/MAR
            cv2.putText(frame, f"EAR: {ear:.2f}", (300, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            cv2.putText(frame, f"MAR: {mar:.2f}", (300, 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

        cv2.imshow("Driver Drowsiness Detection", frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

finally:
    cap.release()
    cv2.destroyAllWindows()

    df = pd.DataFrame(log_data)
    df.to_csv("drowsiness_log.csv", index=False)
    print("[INFO] Log saved → drowsiness_log.csv")

    total_time = round(time.time() - start_time, 2)
    print(f"\n===== 📋 Trip Summary =====")
    print(f"Duration: {total_time} sec | Drowsiness Alerts: {DROWSY_COUNT} | Yawn Alerts: {YAWN_COUNT}")
    genai_alert(f"Trip Summary: Duration {total_time} seconds. Drowsiness alerts {DROWSY_COUNT}. Yawning alerts {YAWN_COUNT}.")


[INFO] Loading face detector and landmark predictor...
[INFO] Starting video stream...
[INFO] Listening for questions... Say 'hey system' to activate.
[INFO] Listening for questions... Say 'hey system' to activate.
[INFO] Listening for questions... Say 'hey system' to activate.
[INFO] Listening for questions... Say 'hey system' to activate.
[INFO] Listening for questions... Say 'hey system' to activate.
⚠️ ALERT: Drowsiness detected! Count: 1


Exception in thread Thread-8 (speak):
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "C:\Users\hp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\ipykernel\ipkernel.py", line 772, in run_closure
Exception in thread Thread-9 (speak):
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "C:\Users\hp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\ipykernel\ipkernel.py", line 772, in run_closure
    _threading_Thread_run(self)
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n

😮 ALERT: Yawning detected! Count: 1
😮 ALERT: Yawning detected! Count: 2


Exception in thread Thread-10 (speak):
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "C:\Users\hp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\ipykernel\ipkernel.py", line 772, in run_closure
    _threading_Thread_run(self)
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\hp\AppData\Local\Temp\ipykernel_18744\2688063776.py", line 27, in speak
  File "C:\Users\hp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pyttsx3\engine.py", line 219, in runAndWait
    raise RuntimeError(msg)
RuntimeError: run loop already started


😮 ALERT: Yawning detected! Count: 3
😮 ALERT: Yawning detected! Count: 4
😮 ALERT: Yawning detected! Count: 5
😮 ALERT: Yawning detected! Count: 6
😮 ALERT: Yawning detected! Count: 7
😮 ALERT: Yawning detected! Count: 8


...

⚠️ ALERT: Drowsiness detected! Count: 9
⚠️ ALERT: Drowsiness detected! Count: 10
[INFO] Listening for questions... Say 'hey system' to activate.


Exception in thread Thread-368 (speak):
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "C:\Users\hp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\ipykernel\ipkernel.py", line 772, in run_closure
    _threading_Thread_run(self)
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\hp\AppData\Local\Temp\ipykernel_18744\2688063776.py", line 27, in speak
  File "C:\Users\hp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pyttsx3\engine.py", line 219, in runAndWait
    raise RuntimeError(msg)
RuntimeError: run loop already started

⚠️ ALERT: Drowsiness detected! Count: 11


Exception in thread Thread-369 (speak):
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "C:\Users\hp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\ipykernel\ipkernel.py", line 772, in run_closure
    _threading_Thread_run(self)
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\hp\AppData\Local\Temp\ipykernel_18744\2688063776.py", line 27, in speak
  File "C:\Users\hp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pyttsx3\engine.py", line 219, in runAndWait
    raise RuntimeError(msg)
RuntimeError: run loop already started

⚠️ ALERT: Drowsiness detected! Count: 12
[INFO] Listening for questions... Say 'hey system' to activate.
[INFO] Listening for questions... Say 'hey system' to activate.
[INFO] Log saved → drowsiness_log.csv

===== 📋 Trip Summary =====
Duration: 82.88 sec | Drowsiness Alerts: 12 | Yawn Alerts: 342
[INFO] Listening for questions... Say 'hey system' to activate.
[INFO] Listening for questions... Say 'hey system' to activate.
[INFO] Listening for questions... Say 'hey system' to activate.
[INFO] Listening for questions... Say 'hey system' to activate.
[INFO] Listening for questions... Say 'hey system' to activate.
[INFO] Listening for questions... Say 'hey system' to activate.
[INFO] Listening for questions... Say 'hey system' to activate.
[INFO] Listening for questions... Say 'hey system' to activate.
[INFO] Listening for questions... Say 'hey system' to activate.
[INFO] Listening for questions... Say 'hey system' to activate.
[INFO] Listening for questions... Say 'hey system' to activate